<a href="https://colab.research.google.com/github/rosadigital/AI_RAG_llama_Engl/blob/main/colab_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar o LlamaIndex
!pip install llama-index

# Instalar o Langchain
!pip install langchain

# Instalar o HuggingFace
!pip install transformers

# Instalar outras dependências
!pip install sentence-transformers
!pip install llama-index-embeddings-langchain
!pip install python-docx
! pip install llama-index-llms-huggingface
! pip install docx2txt

In [2]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM


/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).

In [4]:
documents=SimpleDirectoryReader("pdfs").load_data()


In [5]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""

In [6]:
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate


Looking in indexes: https://pypi.org/simple/


In [8]:
import torch

# llm = HuggingFaceLLM(
#     context_window=4096,
#     max_new_tokens=256,
#     generate_kwargs={"temperature": 0.4, "do_sample": False},
#     system_prompt=system_prompt,
#     tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
#     model_name="meta-llama/Llama-2-7b-chat-hf",
#     device_map="auto",
#     # # uncomment this if using CUDA to reduce memory usage
#     model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True}

# )

llm = HuggingFaceLLM(
    context_window=1024,  # Reduzir o tamanho da janela de contexto
    max_new_tokens=128,  # Reduzir o número máximo de novos tokens gerados
    generate_kwargs={"temperature": 0.4, "do_sample": False},
    system_prompt=system_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True}
    model_kwargs={"torch_dtype": torch.float16}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
embed_model=LangchainEmbedding(HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [10]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-10-f6dd2f399444>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [11]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [12]:
query_engine=index.as_query_engine()

In [13]:
torch.cuda.empty_cache()


In [14]:
response=query_engine.query("what is the population of Brazil?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [15]:
response.response.strip()

"Brazil's population is approximately 212 million people, according to the most recent estimate from the Brazilian Institute of Geography and Statistics (IBGE) in 2020."

In [16]:
questions = [
    "What is ISS?",
    "What taxes exist in Brazil?",
    "What is IRPJ",
    "How has the STF's decision regarding ICMS rates affected Brazilian companies and the Public Treasury?"
]
for question in questions:
    response2=query_engine.query(question)
    print(question)
    print(response2.response.strip())
    print('-'*10)

What is ISS?
ISS stands for Integrated Service System, which is a system used by the Brazilian Federal Revenue Service (RFB) to collect and process information related to the provision of services, including the Municipal Service Tax (ISS). The ISS is a tax levied on the provision of services listed by Supplementary Law 116/2003, and the tax rates may vary between 2% and 5%, depending on the type of service. The municipality to which the ISS must be collected also changes depending on the nature of the service performed.

In addition to the ISS,
----------
What taxes exist in Brazil?
Based on the updated context, the following taxes exist in Brazil:

1. Corporate income tax (CIT)
2. Withholding tax (WHT) on dividends
3. Indirect taxes, including:
a. Consolidated tax on goods and services (ICMS)
b. Social security/pension reform
4. Transfer pricing tax
5. Value-added tax (VAT)
6. Excise tax
7. Customs duties
8. Stamp duty

Additionally, the following taxes were mentioned in the updated 